# Final Project Overview
Your boss asked you to compile the list of the top 10 largest banks in the world ranked by market capitalization in billion USD. Further, you need to transform the data and store it in USD, GBP, EUR, and INR per the exchange rate information made available to you as a CSV file. You should save the processed information table locally in a CSV format and as a database table. Managers from different countries will query the database table to extract the list and note the market capitalization value in their own currency.

## Direction
1. Write a function to extract the tabular information from the given URL under the heading By Market Capitalization, and save it to a data frame.
2. Write a function to transform the data frame by adding columns for Market Capitalization in GBP, EUR, and INR, rounded to 2 decimal places, based on the exchange rate information shared as a CSV file.
3. Write a function to load the transformed data frame to an output CSV file.
4. Write a function to load the transformed data frame to an SQL database server as a table.
5. Write a function to run queries on the database table.
6. Run the following queries on the database table:
   - Extract the information for the London office, that is Name and MC_GBP_Billion
   - Extract the information for the Berlin office, that is Name and MC_EUR_Billion
   - Extract the information for New Delhi office, that is Name and MC_INR_Billion
8. Write a function to log the progress of the code.
9. While executing the data initialization commands and function calls, maintain appropriate log entries.

importing packages/libraries

In [211]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime
import sqlite3

In [212]:
url = 'https://web.archive.org/web/20230908091635 /https://en.wikipedia.org/wiki/List_of_largest_banks'
output_CSV_path = './Largest_banks_data.csv'
db_table_name = 'Largest_banks'
db_name = 'Banks.db'
log_file = 'code_log.txt'
exchange_rates = 'exchange_rate.csv'

In [213]:
def log_info(message):
    ''' This function logs the messages to a log file with a timestamp.
    The log file is created if it does not exist.
    '''
    file = 'code_log.txt'
    timeStamp_format = '%Y-%m-%d %H:%M:%S'
    timeStamp = datetime.now().strftime(timestamp_format)
    with open(file, 'a+') as f:
        f.write(f'{timeStamp} - {message}\n')

In [214]:
def extract(url, table_attribs):
    '''
    Scraping data from provided URL, parsing it and converting it into
    a DataFrame.
    '''
    response = requests.get(url)
    log_info(f'URL Fetched: status code is {response.status_code}')
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        table = soup.find_all('table')[0]
        rows = table.find_all('tr')
        rows = rows[1:]
        data = {}
        # data['rank'] = []
        data[table_attribs[0]] = []
        data[table_attribs[1]] = []
        for row in rows:
            # rank = int(row.find_all('td')[0].text.strip())
            bank_name = row.find_all('td')[1].find_all('a')[-1].text
            market_cap = float(row.find_all('td')[2].text.strip())
            # data['rank'].append(rank)
            data[table_attribs[0]].append(bank_name)
            data[table_attribs[1]].append(market_cap)
        df = pd.DataFrame(data)
        log_info('Extraction: data extracted and loaded into a DataFrame')
        return df

In [215]:
def transform(df, csv_path):
    ''' This function accesses the CSV file for exchange rate
    information, and adds three columns to the data frame, each
    containing the transformed version of Market Cap column to
    respective currencies
    '''
    ex_df = pd.read_csv(exchange_rates, index_col='Currency') # Converting currency into a DF
    df['MC_EUR_Billion'] = df['MC_USD_Billion'] * ex_df.loc['EUR','Rate']
    df['MC_GBP_Billion'] = df['MC_USD_Billion'] * ex_df.loc['GBP','Rate']
    df['MC_INR_Billion'] = df['MC_USD_Billion'] * ex_df.loc['INR','Rate']
    log_info('Transformation: New currency added for EUR, GBP and INR')
    return df

In [216]:
def load_to_csv(df, output_path):
    ''' This function saves the final data frame as a CSV file in
    the  provided path. Function returns nothing.'''
    df.to_csv(output_path, index = False)
    log_info('CSV file created: DataFramewritten in CSV file')
    

In [217]:
def load_to_db(df, sql_connection, table_name):
    ''' This function saves the final data frame to a database
    table with the provided name. Function returns nothing.'''
    try:
        df.to_sql(table_name, sql_connection, if_exists = 'replace', index = False)
    except:
        print("Error encountered while accessing DB")
    finally:
        log_info('CSV file created: DataFramewritten in CSV file')
        sql_connection.close()

In [218]:
def run_query(query_statement, sql_connection):
    ''' This function runs the query on the database table and
    prints the output on the terminal. Function returns nothing. '''
    db_df = pd.read_sql(query_statement,sql_connection)
    sql_connection.close()
    print(db_df)

In [220]:
# Extraction
df = extract(url,['Name', 'MC_USD_Billion'])
# Transformation
new_df = transform(df, exchange_rates)
new_df_rounded = new_df.round(2)
# Loading to CSV
load_to_csv(new_df_rounded,output_CSV_path)
# Loading to SQL
sql_connection = sqlite3.connect(db_name)
load_to_db(new_df_rounded, sql_connection, db_table_name)
# Querying
sql_connection = sqlite3.connect(db_name)
query = f'Select * FROM {db_table_name}'
run_query(query, sql_connection)
# run_query('SELECT Name from Largest_banks LIMIT 5', sql_connection)
sql_connection.close()

                                      Name  MC_USD_Billion  MC_EUR_Billion  \
0                           JPMorgan Chase          432.92          402.62   
1                          Bank of America          231.52          215.31   
2  Industrial and Commercial Bank of China          194.56          180.94   
3               Agricultural Bank of China          160.68          149.43   
4                                HDFC Bank          157.91          146.86   
5                              Wells Fargo          155.87          144.96   
6                        HSBC Holdings PLC          148.90          138.48   
7                           Morgan Stanley          140.83          130.97   
8                  China Construction Bank          139.82          130.03   
9                            Bank of China          136.81          127.23   

   MC_GBP_Billion  MC_INR_Billion  
0          346.34        35910.71  
1          185.22        19204.58  
2          155.65        16138.75